<a href="https://colab.research.google.com/github/IramKayani/ATM-Machine-456/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
! pip install langchain-google-genai langchain-Community Chromadb pyPDF -q


In [53]:
from google.colab import userdata
Key=userdata.get('API-KEY')
from langchain_google_genai import GoogleGenerativeAI
llm=GoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=Key
)
import os
os.environ["google-API-Key"]=Key

In [54]:
from langchain_community.document_loaders import PyPDFLoader
Document=PyPDFLoader("SOP 2022.pdf")
docs=Document.load()

In [56]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=Key)

In [57]:
from langchain.vectorstores import Chroma
db=Chroma.from_documents(docs,embeddings)

In [58]:
from langchain.chains import retrieval
retrieval=db.as_retriever(search_type="similarity",search_kwargs={"k":10})

In [59]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [60]:
sytem_Prompt = "You are an assistance if you dont know the answer Say sorry"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", sytem_Prompt),
        ("human", "{context}{input}"),  # Add {context} here to include the document content
    ]
)

In [61]:
Question = create_stuff_documents_chain(llm,prompt)
Rag=create_retrieval_chain(retrieval,Question)

In [64]:
response = Rag.invoke({"input": "What is Retention Money?"})
print(response)

{'input': 'What is Retention Money?', 'context': [Document(metadata={'page': 66, 'page_label': '67', 'source': 'SOP 2022.pdf'}, page_content="Viewed by  118636irum at  030934 Feb 23\nViewed by  118636irum at  030934 Feb 23\nViewed by  118636irum at  030934 Feb 23\nAgreement No.__________________ \nby the client, case for part payment (not more than 50%) of final bill may be processed by \nDir Projs on min sheet for approval by CFA.  Payment will be made by the concerned PMU \nafter issuance of payment auth by CFO NLC. Hand receipts for rel of Retention money will \nbe processed after defect liability pd (DLP).  Any delay in prep and submission of \nbills/interim bills markup etc, will not be attributed to NLC as such the sub  contractor will \nnot claim compen.  \n1.15 Retention Money.  The deduction of retention money from sub-contractor will be as per NLC CA \nwith the Client ( back-to-back), however, min 5% will be deducted. During currency of work, 50% \nof retention money may be r

In [63]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000
)
